# Middleware in LangChain 
mechanism that allows you to intercept, modify, or extend the behavior of chains and tools as they process inputs and outputs. It acts as a layer between the user and the core logic, enabling tasks like logging, validation, or custom processing without changing the main workflow.

## Example
Suppose you want to check if a user’s input contains any banned words before processing it in LangChain. You can use middleware to scan the input for banned words. If any are found, the middleware can block the request or return a warning, otherwise it lets the input continue to the main logic. This helps keep your application safe and clean without changing the core processing code.

In [3]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model('groq:qwen/qwen3-32b')
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000023178DCAFD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023178DCB9D0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

### Summarization MiddleWare


In [4]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import SystemMessage, HumanMessage

agent = create_agent(
    model='groq:qwen/qwen3-32b',
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model='groq:qwen/qwen3-32b',
            trigger = ("messages",10),
            keep = ("messages",4)
        )
    ]
)

In [5]:
### Run with thread ID
config={"configurable":{"thread_id":"test_thread"}}

In [6]:
question = [
    "what is 2+2",
    "what is 3+3",
    "what is 4+4",
    "what is 5+5",
    "what is 6+6",
    "what is 7+7",
    "what is 8+8",
    # "what is 9+9",
]
for q in question:
    response = agent.invoke({"messages" : [HumanMessage(content=q)]},config)
    print(f"Messages: {response}")
    print(f"Number of messages: {len(response['messages'])}")

Messages: {'messages': [HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}, id='2aa8111a-39e8-49ad-8868-79c87e029c42'), AIMessage(content='<think>\nOkay, the user is asking "what is 2+2". That\'s a basic arithmetic question. Let me start by recalling that addition is one of the fundamental operations in mathematics. When you add two numbers together, you\'re combining their quantities. So 2 plus 2 would be 4. But wait, maybe I should double-check that. Let me count: 2 objects plus another 2 objects make a total of 4 objects. Yes, that seems right.\n\nI wonder if there\'s any context where 2+2 could mean something else. In most everyday situations, it\'s straightforward addition. But in some specialized fields, like modular arithmetic or different number bases, the result might vary. For example, in base 3, 2+2 would be 11 (which is 4 in decimal). However, unless specified otherwise, the default assumption is base 10. Since the question doesn\'t mention any s

### Token Size

In [12]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import SystemMessage, HumanMessage

@tool
def search_hotel(city:str) -> str:
    """"Search for hotels in a city and return the results as a string."""
    # In a real implementation, this function would call an external API to search for hotels.
    # For this example, we'll just return a hardcoded string.
    return f"""Hotels in city : {city}:
    1. Hotel A\n2. Hotel B\n3. Hotel C.
    2. Hotel D\n3. Hotel E.
    3. Hotel F\n4. Hotel G\n5. Hotel H.
    """

agent = create_agent(
    model = 'groq:qwen/qwen3-32b',
    tools = [search_hotel],
    checkpointer = InMemorySaver(),
    middleware = [SummarizationMiddleware(
        model = 'groq:qwen/qwen3-32b',
        trigger = ("tokens",550),
        keep = ("tokens",200),
    ),
    ]
)
config = {"configurable":{"thread_id":"test_thread"}}

def count_tokesn(messages):
    total = sum(len(str(m.content)) for m in messages)
    return total // 4

In [13]:
cities = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"]
for city in cities:
    response = agent.invoke({"messages" : [HumanMessage(content=f"Search for hotels in {city}")]},config)

    tokens = count_tokesn(response['messages'])
    print(f"{city} - Tokens: {tokens} tokens,{len(response['messages'])} messages")
    print(f"{(response['messages'])}")

New York - Tokens: 102 tokens,4 messages
[HumanMessage(content='Search for hotels in New York', additional_kwargs={}, response_metadata={}, id='3a773193-bd39-496b-afdc-3f6df7200327'), AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user wants to search for hotels in New York. Let me check the available tools. There\'s a function called search_hotel that takes a city parameter. The required parameter is city, and it\'s a string. Since the user specified New York, I need to call this function with "New York" as the city argument. I\'ll make sure to format the tool call correctly within the XML tags as specified.\n', 'tool_calls': [{'id': '8pbpj483n', 'function': {'arguments': '{"city":"New York"}', 'name': 'search_hotel'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 160, 'total_tokens': 272, 'completion_time': 0.165327871, 'completion_tokens_details': {'reasoning_tokens': 86}, 'prompt_time': 0.006561001, 'p

KeyboardInterrupt: 

### Fraction

In [ ]:
# Example: SummarizationMiddleware with Fraction Trigger for Tokens
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage

# Fraction trigger: summarize when tokens exceed 0.005 of max tokens (e.g., 0.5%)
fraction = 0.005
max_tokens = 1000  # Example max tokens
trigger_tokens = int(max_tokens * fraction)
keep_tokens = int(max_tokens * 0.2)  # Keep 20% of max tokens

agent = create_agent(
    model='groq:qwen/qwen3-32b',
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model='groq:qwen/qwen3-32b',
            trigger=("tokens", trigger_tokens),
            keep=("tokens", keep_tokens),
        )
]
)

questions = [
    "What is 1/2 plus 1/3?",
    "What is 2+2?",
    "Calculate 3/4 times 2/5.",
    "Add 5 and 7."
    # Add more questions to reach token threshold
    "Explain the process of photosynthesis in detail.",
    "Describe the history of the Roman Empire.",
    "What are the main causes of climate change?"
]
config = {"configurable": {"thread_id": "test_thread"}}
for q in questions:
    response = agent.invoke({"messages": [HumanMessage(content=q)]},config)
    print(f"Messages: {response}")

Messages: {'messages': [HumanMessage(content='What is 1/2 plus 1/3?', additional_kwargs={}, response_metadata={}, id='1a3aab8c-c7ef-4e99-9361-25f175c34a51'), AIMessage(content="<think>\nOkay, so I need to figure out what 1/2 plus 1/3 is. Hmm, adding fractions. Let me think. I remember that to add fractions, they have to have the same denominator. Right? If the denominators are different, you can't just add the numerators directly. So first, I need to find a common denominator for 1/2 and 1/3. \n\nThe denominators here are 2 and 3. What's the least common denominator of 2 and 3? Well, the least common multiple of 2 and 3. Let me think. Multiples of 2 are 2, 4, 6, 8, 10, 12... and multiples of 3 are 3, 6, 9, 12... So the smallest common multiple is 6. So 6 is the least common denominator. \n\nOkay, so I need to convert both fractions to have 6 as the denominator. Starting with 1/2. To turn 2 into 6, I multiply the denominator by 3. So I have to do the same to the numerator. So 1 times 3 